In [2]:
import os
import math
import time
import magenta.music as mm
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from magenta.music import DEFAULT_QUARTERS_PER_MINUTE
from magenta.music.protobuf.generator_pb2 import GeneratorOptions
from magenta.music.protobuf.music_pb2 import NoteSequence
from visual_midi import Plotter

In [3]:
def generate(bundle_name, seq_generator, gen_id, primer, qpm = DEFAULT_QUARTERS_PER_MINUTE, 
             total_steps = 64, temp = 1.0, beam_size = 1, branch_factor = 1, steps_per_iteration = 1):
    
    mm.notebook_utils.download_bundle(bundle_name, "bundles")
    bundle = sequence_generator_bundle.read_bundle_file(os.path.join("bundles", bundle_name))
    
    generator_map = seq_generator.get_generator_map()
    generator = generator_map[gen_id](checkpoint = None, bundle = bundle)
    generator.initialize()
    
    if primer:
        primer_seq = mm.midi_io.midi_file_to_note_sequence(primer)
    else:
        primer_seq = NoteSequence()
        
    if primer_seq.tempos:
        if len(primer_seq.tempos) > 1:
            raise Exception('Monophonic Melody Generation. No support for multiple tempos')
        qpm = primer_seq.tempos[0].qpm
        
    secs_per_step = 60.0 / (qpm * getattr(generator, 'steps_per_quarter', 4))
    
    primer_seq_steps = math.ceil(primer_seq.total_time / secs_per_step)
    primer_seq_time = primer_seq_steps * secs_per_step
    primer_end_ad = (0.00001 if primer_seq_time > 0 else 0)
    primer_start_time = 0
    primer_end_time = primer_start_time + primer_seq_time - primer_end_ad
    
    gen_steps = total_steps - primer_seq_steps
    
    if gen_steps <= 0:
        raise Exception('Total steps too small : {}. Need at least 1 bar bigger than primer : {}'.format(
        total_steps, primer_seq_steps))
    
    gen_time = gen_steps * secs_per_step
    gen_start_time = primer_end_time
    gen_end_time = gen_start_time + gen_time + primer_end_ad
    
    print('Primer start : {} \t Primer end : {}'.format(primer_start_time, primer_end_time))
    print('Generation start : {} \t Generation end : {}'.format(gen_start_time, gen_end_time))
    
    generator_options = GeneratorOptions()
    generator_options.args['temperature'].float_value = temp
    generator_options.args['beam_size'].int_value = beam_size
    generator_options.args['branch_factor'].int_value = branch_factor
    generator_options.args['steps_per_iteration'].int_value = steps_per_iteration
    generator_options.generate_sections.add(start_time = gen_start_time, end_time = gen_end_time)
    
    seq = generator.generate(primer_seq, generator_options)
    
    # Write midi file
    date_time = time.strftime('%Y-%m-%d_%H%M%S')
    gen_name = str(generator.__class__).split('.')[2]
    midi = '%s_%s_%s.mid' % (gen_name, gen_id, date_time)
    midi_path = os.path.join('output', midi)
    mm.midi_io.note_sequence_to_midi_file(seq, midi_path)
    print('Generated midi : {}'.format(os.path.abspath(midi_path)))
    
    # Generate plot
    date_time = time.strftime('%Y-%m-%d_%H%M%S')
    gen_name = str(generator.__class__).split('.')[2]
    plot = '%s_%s_%s.html' % (gen_name, gen_id, date_time)
    plot_path = os.path.join('output', plot)
    
    pretty_midi = mm.midi_io.note_sequence_to_pretty_midi(seq)
    plotter = Plotter()
    plotter.show(pretty_midi, plot_path)
    print('Generated plot : {}'.format(os.path.abspath(plot_path)))
    
    return seq

In [28]:
sequence = generate('basic_rnn.mag', melody_rnn_sequence_generator, 
                    'basic_rnn', primer = 'primers/Fur_Elisa_Beethoveen_Monophonic.mid',
                    total_steps = 64, temp = 0.9)

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from C:\Users\siddh\AppData\Local\Temp\tmpml5dilpn\model.ckpt
Primer start : 0 	 Primer end : 1.6463307499999995
Generation start : 1.6463307499999995 	 Generation end : 11.707311999999995
INFO:tensorflow:Beam search yields sequence with log-likelihood: -114.740669 
Generated midi : C:\Users\siddh\Documents\GitHub\magenta_project\output\melody_rnn_basic_rnn_2020-12-05_054555.mid
Generated plot : C:\Users\siddh\Documents\GitHub\magenta_project\output\melody_rnn_basic_rnn_2020-12-05_054555.html


In [29]:
sequence = generate('lookback_rnn.mag', melody_rnn_sequence_generator,
                    'lookback_rnn', primer = 'primers/Fur_Elisa_Beethoveen_Monophonic.mid',
                    total_steps = 64, temp = 1.1)

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from C:\Users\siddh\AppData\Local\Temp\tmpjcn82waa\model.ckpt
Primer start : 0 	 Primer end : 1.6463307499999995
Generation start : 1.6463307499999995 	 Generation end : 11.707311999999995
INFO:tensorflow:Beam search yields sequence with log-likelihood: -89.669540 
Generated midi : C:\Users\siddh\Documents\GitHub\magenta_project\output\melody_rnn_lookback_rnn_2020-12-05_054818.mid
Generated plot : C:\Users\siddh\Documents\GitHub\magenta_project\output\melody_rnn_lookback_rnn_2020-12-05_054818.html


In [32]:
sequence = generate('attention_rnn.mag', melody_rnn_sequence_generator,
                    'attention_rnn', primer = 'primers/Fur_Elisa_Beethoveen_Monophonic.mid',
                    total_steps = 128, temp = 1.1)

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from C:\Users\siddh\AppData\Local\Temp\tmp78v1yg4m\model.ckpt
Primer start : 0 	 Primer end : 1.6463307499999995
Generation start : 1.6463307499999995 	 Generation end : 23.414623999999993
INFO:tensorflow:Beam search yields sequence with log-likelihood: -224.228256 
Generated midi : C:\Users\siddh\Documents\GitHub\magenta_project\output\melody_rnn_attention_rnn_2020-12-05_055245.mid
Generated plot : C:\Users\siddh\Documents\GitHub\magenta_project\output\melody_rnn_attention_rnn_2020-12-05_055245.html
